# UNIFAQ TEST

In [3]:
from IPython.display import display, Math, Latex
from IPython.core.display import display_html

In [6]:
from __future__ import division, print_function
import pandas, numpy as np, itertools, six

class UNIFAQLoader():
    def __init__(self):
        self.df_components = pandas.read_excel('Horstmann.xlsx','Components')
        self.df_groups = pandas.read_excel('Horstmann.xlsx','Groups')
        self.df_interaction = pandas.read_excel('Horstmann.xlsx','InteractionParameters')
        self.df_interaction = self.df_interaction.fillna(0.0)

    def get_component(self, CAS):
        mask = (self.df_components['CAS-nr.'] == CAS)
        if np.sum(mask) == 0: raise KeyError("Cannot match CAS #: "+CAS)
        groups_string = self.df_components.loc[mask,'increments [counter * sub group number]'].iloc[0]
        groups = []
        for grouping in groups_string.replace(u'\xd7','*').split(';'):
            count, sub_group_index = grouping.strip().split('*')
            groups.append(dict(count=int(count), sub_group_index = int(sub_group_index)))
        return groups

    def get_group_parameters(self, sub_group_index):
        mask = (self.df_groups['sub group index'] == sub_group_index)
        if np.sum(mask) == 0: raise KeyError("Cannot match sub group index #: " + sub_group_index)
        return dict(R_k = self.df_groups.loc[mask,'Rk'].iloc[0], 
                    Q_k = self.df_groups.loc[mask,'Qk'].iloc[0],
                    main_group_index = self.df_groups.loc[mask,'main group index'].iloc[0])

    def get_interaction_parameters(self, main_group_index1, main_group_index2):
        if main_group_index1 == main_group_index2: return dict(aij=0,aji=0,bij=0,bji=0,cij=0,cji=0)
        if main_group_index1 < main_group_index2:
            mask = ((self.df_interaction['i'] == main_group_index1) & (self.df_interaction['j'] == main_group_index2))
            if np.sum(mask) == 0: raise KeyError("Cannot match i= " + main_group_index1 + ' j=' + main_group_index2)
            return dict(aij = self.df_interaction.loc[mask,'aij / K'].iloc[0],
                        aji = self.df_interaction.loc[mask,'aji / K'].iloc[0],
                        bij = self.df_interaction.loc[mask,'bij'].iloc[0],
                        bji = self.df_interaction.loc[mask,'bji'].iloc[0],
                        cij = self.df_interaction.loc[mask,'cij / K-1'].iloc[0],
                        cji = self.df_interaction.loc[mask,'cji / K-1'].iloc[0],
                        )
        else:
            mask = ((self.df_interaction['j'] == main_group_index1) & (self.df_interaction['i'] == main_group_index2))
            if np.sum(mask) == 0: raise KeyError("Cannot match i= " + main_group_index1 + ' j=' + main_group_index2)
            return dict(aij = self.df_interaction.loc[mask,'aji / K'].iloc[0],
                        aji = self.df_interaction.loc[mask,'aij / K'].iloc[0],
                        bij = self.df_interaction.loc[mask,'bji'].iloc[0],
                        bji = self.df_interaction.loc[mask,'bij'].iloc[0],
                        cij = self.df_interaction.loc[mask,'cji / K-1'].iloc[0],
                        cji = self.df_interaction.loc[mask,'cij / K-1'].iloc[0]
                        )


class UNIFAQMixture(object):
    def __init__(self, components, z, T):
        self.pures = self.decompose_components(components)
        self.interaction = self.load_interaction_parameters()
        self.z = np.array(z)
        self.T = T
        self.calculate_pures(self.z)
        self.calculate_subgroup_counts()

    def calculate_pures(self, z):
        self.r = np.zeros_like(z)
        self.q = np.zeros_like(z)
        self.l = np.zeros_like(z)
        self.theta = np.zeros_like(z)
        self.phi = np.zeros_like(z)
        self.ln_phi_C = np.zeros_like(z)
        self.Ngroups = np.zeros_like(z, dtype = int)
        for i in range(len(z)):
            self.r[i] = sum([g['count']*g['R_k'] for g in self.pures[i]])
            self.q[i] = sum([g['count']*g['Q_k'] for g in self.pures[i]])
            self.Ngroups[i] = sum([g['count'] for g in self.pures[i]])
        for i in range(len(z)):
            self.phi[i] = (self.r[i]*self.z[i])/np.sum(self.z*self.r)
            self.theta[i] = (self.q[i]*self.z[i])/np.sum(self.z*self.q)
            self.l[i] = 10.0/2.0*(self.r[i] - self.q[i]) - (self.r[i]-1)
        self.ln_gamma_C = np.log(self.phi/self.z) + 10.0/2.0*self.q*np.log(self.theta/self.phi) + self.l - self.phi/self.z*sum(self.z*self.l)

    def calculate_subgroup_counts(self):
        self.subgroup_counts = {}
        for sgi in self.sub_groups_present:
            sgc = np.zeros_like(self.z)
            for i in range(len(self.z)):
                for group in self.pures[i]:
                    if group['sub_group_index'] == sgi:
                        sgc[i] = group['count']
            self.subgroup_counts[sgi] = sgc

    def decompose_components(self, components):
        self.UNIFAQ = UNIFAQLoader()
        pures = []
        for CAS in components:
            pure = []
            for group in self.UNIFAQ.get_component(CAS):
                group_params = self.UNIFAQ.get_group_parameters(group['sub_group_index'])
                group.update(group_params)
                pure.append(group)
            pures.append(pure)
        return pures

    def load_interaction_parameters(self):
        # Get all the sub- and main-group indices
        indices = []
        for pure in self.pures:
            indices += [(g['sub_group_index'], g['main_group_index']) for g in pure]
        # Keep only unique entries
        indices = list(set(indices))
        
        # Mapping from sub group index to main group index
        mgi_mapping = {sgi:mgi for sgi, mgi in indices}
        # Unpack the indices
        sub_group_indices, main_group_indices = zip(*indices)
        # Store the list of subgroups that are present
        self.sub_groups_present = sub_group_indices
        # Get all the permutations of sub-group interactions
        interaction = {}
        for sgi1,sgi2 in itertools.product(sub_group_indices, repeat=2):
            # Map to main group index
            i = mgi_mapping[sgi1]
            j = mgi_mapping[sgi2]
            
            interaction[(sgi1,sgi2)] = self.UNIFAQ.get_interaction_parameters(i, j)
            interaction[(sgi2,sgi1)] = self.UNIFAQ.get_interaction_parameters(j, i)
        return interaction

    def pure_i(self, i):
        return dict(q = self.q[i], r = self.r[i], phi = self.phi[i], theta = self.theta[i], l = self.l[i])

    def X_pure_i(self, i, as_dict = False):
        """ List of mole fractions of sub group indices """
        sub_group_indices,counts,Q = zip(*[(g['sub_group_index'],g['count'],g['Q_k']) for g in self.pures[i]])
        total = sum(counts)
        data = [(sgi, count/total, Q) for sgi, count, Q in zip(sub_group_indices,counts, Q)]
        if not as_dict:
            return data
        else:
            return {sgi:(theta,Q) for sgi,theta,Q in data}

    def theta_pure_i(self, i, as_dict = False):
        """ List of theta values for the sub groups in pure fluid index i """
        sgi, X, Q = zip(*self.X_pure_i(i))
        total = sum([_X*_Q for _X,_Q in zip(X, Q)])
        data = [(sgi, X*Q/total) for sgi, X, Q in zip(sgi, X, Q)]
        if not as_dict:
            return data
        else:
            return {sgi:theta for sgi,theta in data}

    def Psi_mn(self, sgi1, sgi2):
        params = self.interaction[(sgi1,sgi2)]
        return np.exp(-(params['aij'] + params['bij']*self.T + params['cij']*self.T**2)/self.T)

    def lnGamma_pure_i(self, i):
        sgi, X, Q = zip(*self.X_pure_i(i))
        sgi, theta = zip(*self.theta_pure_i(i))
        data = {}
        for k, Qk in zip(sgi,Q):
            sum1 = sum([thetam*self.Psi_mn(m,k) for m, thetam in zip(sgi, theta)])
            s = 1-np.log(sum1)
            sum2 = 0
            for m,thetam in zip(sgi,theta):
                s -= thetam*self.Psi_mn(k,m)/sum([thetan*self.Psi_mn(n,m) for n,thetan in zip(sgi,theta)])
            data[k] = Qk*s
        return data

    def lnGamma_mix(self):
        collec = {}
        for pure in self.pures:
            for g in pure:
                sgi, count, Q = g['sub_group_index'],g['count'],g['Q_k']
                if sgi not in collec:
                    collec[sgi] = dict(count = count, Q = Q)
                else:
                    collec[sgi]['count'] += count
        
        # Total number of groups, weighted by the mole fraction
        group_sum = np.sum(self.z*self.Ngroups)
        print(group_sum)

        X = {}
        for sgi in self.sub_groups_present:
            X[sgi] = sum([z*c for z,c in zip(self.z, self.subgroup_counts[sgi])])/group_sum

        theta = {}
        # Denominator of summation for theta
        theta_summer = sum([X[sgi]*collec[sgi]['Q'] for sgi in self.sub_groups_present])

        for sgi in self.sub_groups_present:
            theta[sgi] = X[sgi]*collec[sgi]['Q']/theta_summer

        lnGamma = {}
        for k in self.sub_groups_present:
            sum1 = sum([theta[m]*self.Psi_mn(m,k) for m in self.sub_groups_present])
            s = 1-np.log(sum1)
            sum2 = 0
            for m in self.sub_groups_present:
                s -= theta[m]*self.Psi_mn(k,m)/sum([theta[n]*self.Psi_mn(n,m) for n in self.sub_groups_present])
            lnGamma[k] = collec[k]['Q']*s
        
        self.lnGamma = lnGamma
        return lnGamma, X, theta

    def ln_gamma_R(self, i):
        summer = 0
        for sgi in self.sub_groups_present:
            for group in self.pures[i]:
                if group['sub_group_index'] == sgi:
                    summer += group['count']*(self.lnGamma[sgi] - self.lnGamma_pure_i(i)[sgi])
        return summer

# Example 8-12 from Poling et al., "The Properties of Gases and Liquids (5th Edition)"
# Coefficients are obtained from the table of Horstmann, as attached in Excel spreadsheet
CAS_Acetone = '67-64-1'
CAS_Pentane = '109-66-0'
z = [0.047, 1-0.047]
T = 307 #[K]
mix = UNIFAQMixture([CAS_Acetone,CAS_Pentane], z, T)
# Collect all the data on the pure fluids
pures = [mix.pure_i(i) for i in range(len(z))]
# Turn a list of dicts of single values into a dict of lists
flattened = {k:[pures[i][k] for i in range(len(z))] for k in pures[0].keys()}

# Display the table of pure fluid values
display(pandas.DataFrame(flattened))

display(Math('\\ln\\gamma_{1}^C = ' + str(mix.ln_gamma_C[0])))
display(Math('\\ln\\gamma_{1}^C = ' + str(mix.ln_gamma_C[1])))

display(Math('X_1^{(1)} = ' + str(mix.X_pure_i(0, as_dict=True)[1][0])))
display(Math('X_{18}^{(1)} = ' + str(mix.X_pure_i(0, as_dict=True)[18][0])))
display(Math('\\theta_1^{(1)} = ' + str(mix.theta_pure_i(0, as_dict=True)[1])))
display(Math('\\theta_{18}^{(1)} = ' + str(mix.theta_pure_i(0, as_dict=True)[18])))
display(Math('\\ln\\Gamma_1^{(1)} = ' + str(mix.lnGamma_pure_i(0)[1])))
display(Math('\\ln\\Gamma_{18}^{(1)} = ' + str(mix.lnGamma_pure_i(0)[18])))

display(Math('X_1^{(2)} = ' + str(mix.X_pure_i(1, as_dict=True)[1][0])))
display(Math('X_{2}^{(2)} = ' + str(mix.X_pure_i(1, as_dict=True)[2][0])))
display(Math('\\theta_1^{(1)} = ' + str(mix.theta_pure_i(1, as_dict=True)[1])))
display(Math('\\theta_{2}^{(1)} = ' + str(mix.theta_pure_i(1, as_dict=True)[2])))
display(Math('\\ln\\Gamma_1^{(2)} = ' + str(mix.lnGamma_pure_i(1)[1])))
display(Math('\\ln\\Gamma_{2}^{(2)} = ' + str(mix.lnGamma_pure_i(1)[2])))

# For the mixture
lnGamma, X, theta = mix.lnGamma_mix()
for i in [1,2,18]:
    display(Math('X_{'+str(i)+'} = ' + str(X[i])))
for i in [1,2,18]:
    display(Math('\\theta_{'+str(i)+'} = ' + str(theta[i])))
for i in [1,2,18]:
    display(Math('\\ln\\Gamma_{'+str(i)+'} = ' + str(lnGamma[i])))

display(Math(r'\ln\gamma_1^{\rm R} = ' + str(mix.ln_gamma_R(0))))
display(Math(r'\ln\gamma_2^{\rm R} = ' + str(mix.ln_gamma_R(1))))

display(Math(r'\gamma_1 = ' + str(np.exp(mix.ln_gamma_R(0) + mix.ln_gamma_C[0]))))
display(Math(r'\gamma_2 = ' + str(np.exp(mix.ln_gamma_R(1) + mix.ln_gamma_C[1]))))

,l,phi,q,r,theta
0,-0.3860,0.032113,2.336,2.5735,0.033576
1,-0.2784,0.967887,3.316,3.8254,0.966424


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>